In [145]:
import tensorflow as tf
import os
import cv2

from tensorflow import keras
import random
from keras.preprocessing.image import ImageDataGenerator

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec
from PIL import Image, ImageOps, ImageFilter, ImageEnhance  #PILLOW
from math import floor, ceil, pi
import numpy as np 
%matplotlib inline

In [146]:
#Funzione per fare mirror orizzonale delle immagini e delle relative maschere
            #immagine, nome immagine, maschera, salvataggio immagine, salvataggio maschera
def mirror_hor(img,name_img,mask,dest_img, dest_mask):
    flip_img = ImageOps.mirror(img)
    flip_mask = ImageOps.mirror(mask)
    dest_img = os.path.join(dest_img,"trasl_"+name_img)
    dest_mask = os.path.join(dest_mask,"trasl_"+name_img)
    flip_img.save(dest_img)
    flip_mask.save(dest_mask)

In [147]:
#Funzione per cambiare il colore di un immagine (la maschera rimane uguale)
def color(img,name_img,mask,dest_img,dest_mask):
    img = img.convert("RGBA")
    factor = random.uniform(0.0,3.0) # fattore random per la modifica del colore
    new_img = ImageEnhance.Color(img).enhance(factor)
    
    #Save
    dest_img = os.path.join(dest_img,"col_"+name_img)
    dest_mask = os.path.join(dest_mask,"col_"+name_img)
    new_img.save(dest_img)
    mask.save(dest_mask)

In [148]:
#Funzione per aggiungere rumore gaussiano alle immagini (le maschere rimangono uguali)
#Aggiunta di pixel di vari colori sull'immagine
def gaussian_noise(image_path,name_img,mask,dest_img,dest_mask):
    img = cv2.imread("..\\original-dataset\\images\\"+name_img) #unico che funziona
   
    # Generate Gaussian noise
    gauss = np.random.normal(0,1,img.size)
    gauss = gauss.reshape(img.shape[0],img.shape[1],img.shape[2]).astype('uint8')
    # Add the Gaussian noise to the image
    img_gauss = cv2.add(img,gauss)
    
    # Display the image
    #cv2.imshow('a',img_gauss)
    #cv2.waitKey(0)
    cv2.imwrite('..\\car-segmentation\\images\\gauss_'+name_img,img_gauss)
    dest_mask = os.path.join(dest_mask,"gauss_"+name_img)
    mask.save(dest_mask)

In [149]:
#Funzione per sfocare o definire un'immagine in modo random (maschere uguali)
def blur(img,name_img,mask,dest_img,dest_mask):
    img = img.convert("RGBA")
    
    factor = random.uniform(-2.0,4.0) # fattore random per la modifica del colore
    new_img = ImageEnhance.Sharpness(img).enhance(factor)
    #new_img.show()
    
    #Save
    dest_img = os.path.join(dest_img,"blur_"+name_img)
    dest_mask = os.path.join(dest_mask,"blur_"+name_img)
    new_img.save(dest_img)
    mask.save(dest_mask)
    

In [150]:
def rotate(img,name_img,mask,dest_img,dest_mask):
    new_img = img.rotate(25,expand=True,fillcolor=(255, 128, 0,0))
    new_img.show()
    
    #Save
    #dest_img = os.path.join(dest_img,"blur_"+name_img)
    #dest_mask = os.path.join(dest_mask,"blur_"+name_img)
    #new_img.save(dest_img)
    #mask.save(dest_mask)
    

In [151]:
#Get Directory of car-segmentation images
parent = os.path.abspath(os.path.join(os.getcwd(), os.pardir)) #parent directory
img_folder = os.path.join(parent, "original-dataset", "images")
masks_folder = os.path.join(parent, "original-dataset", "masks")
destination_img_folder = os.path.join(parent, "car-segmentation", "images") #cartella di destinazione su cui salvare le immagini
destination_mask_folder = os.path.join(parent, "car-segmentation", "masks") #cartella di destinazione su cui salvare le maschere 
all_images = os.listdir(img_folder)
all_masks = os.listdir(masks_folder)

for i in range(1): #len(all_images)
    print(i)
    #immagine
    file = all_images[i] #Nome dell'immagine (e della maschera)
    image_path = os.path.join(img_folder, file)
    img = Image.open(image_path)
    #maschera
    mask_path = os.path.join(masks_folder,file)
    mask = Image.open(mask_path)
    
    #Operazioni
    #1) Mirror orizzonale di ogni immagine
    mirror_hor(img,file,mask,destination_img_folder,destination_mask_folder)
    
    #2) Rumore gaussiano
    gaussian_noise(image_path,file,mask,destination_img_folder,destination_mask_folder)
    
    #3) Colore
    color(img,file,mask,destination_img_folder,destination_mask_folder)
    
    #4) Blur
    blur(img,file,mask,destination_img_folder,destination_mask_folder)
    
    #5) Rotate 30°
    #rotate(img,file,mask,destination_img_folder,destination_mask_folder)
    
  

1
2
3
4
5
6
7
8
9
